In [1]:
from selenium import webdriver
import time
import random as r

import requests
import io
from PIL import Image

# SCRAPPING SENS CRITIQUE

1 - RECUPEREATION DE LA LISTE DE NOM DE FILM

In [2]:
#La liste des liens où on va faire le scrapping
liste_liens = {
    2013 : "https://www.senscritique.com/top/resultats/Les_meilleurs_films_de_2013/173207",
    2014 : "https://www.senscritique.com/top/resultats/Les_meilleurs_films_de_2014/367137",
    2015 : "https://www.senscritique.com/top/resultats/Les_meilleurs_films_de_2015/703337",
    2016 : "https://www.senscritique.com/top/resultats/Les_meilleurs_films_de_2016/1152822",
    2017 : "https://www.senscritique.com/top/resultats/Les_meilleurs_films_de_2017/1522840",
    2018 : "https://www.senscritique.com/top/resultats/Les_meilleurs_films_de_2018/1757790",
    2019 : "https://www.senscritique.com/top/resultats/Les_meilleurs_films_de_2019/2301802"
}


In [3]:
liste_nom_film = []
IS_LINUX = False
chrome = webdriver.Chrome(executable_path="chromedriver" if not IS_LINUX else "./chromedriver_linux")
for i in range(2013, 2020):
    chrome.get(liste_liens[i])
    chrome.execute_script("window.scrollTo(0, document.body.scrollHeight);") 
    time.sleep(1)
    
    all_widgets = chrome.find_elements_by_class_name("d-rubric")
    first_line = all_widgets[0]
    all_first_line_elements = first_line.find_elements_by_class_name('elpo-item')
    
    def extract_title(offer_web_element):
        lst_title = []
        for j in range(len(offer_web_element)):
            d = offer_web_element[j].text
            d = d.split("\n")
            d = d[1]
            d = d.split("("+str(i)+")")
            lst_title.append(d[0][0:-1])
        return lst_title
    
    liste_nom_film.append(extract_title(all_first_line_elements))
    
    
len(liste_nom_film)

7

In [4]:
#On va nettoyer la liste que nous venons de récupérer
for i in range(len(liste_nom_film)):
    for j in range(len(liste_nom_film[i])):
        if " (20" in liste_nom_film[i][j]:
            liste_nom_film[i][j] = liste_nom_film[i][j].split(" (20")[0]

doubles = []
for i in range(len(liste_nom_film)):
    for j in range(len(liste_nom_film[i])):
        film = liste_nom_film[i][j]
        for k in range(7):
            if film in liste_nom_film[k] and k!=i:
                doubles.append([i,j])
for copie in doubles:
    del liste_nom_film[copie[0]][copie[1]]

2 - Récupération des infos pour chaque film

In [5]:
def recuperation_donnee(nom_film):
    link = chrome.find_element_by_link_text(nom_film)
    link.click()
    time.sleep(3)
    all_widgets_desc = chrome.find_elements_by_class_name("pvi-productDetails")
    description = all_widgets_desc[0].find_elements_by_class_name('pvi-productDetails-item')
    a = []
    for elt in description:
        a.append(elt.text)
    type_production = a[0].split(' de ')[0]
    if type_production == "Documentaire":
        style_film = None
        realisateur = ",".join(a[0].split(' de ')[1].split(' et ')).split(",")
        duree = a[1]
        date_de_sortie = a[2]
    else:
        style_film = ",".join(a[1].split(' et ')).split(",")
        realisateur = ",".join(a[0].split(' de ')[1].split(' et ')).split(",")
        duree = a[2]
        date_de_sortie = a[3]
    try:
        acteurs = "".join(chrome.find_elements_by_class_name('pvi-productDetails-workers')[0].text.split('Avec ')).split(',')
    except:
        acteurs = None
    try:
        link = chrome.find_element_by_class_name("d-link")
        link.click()
        synopsis = chrome.find_elements_by_xpath('/html/body/div[3]/div[2]/div[3]/div/section[1]/p[2]')[0].text.split(' Fermer')[0]
        if synopsis=='':
            while synopsis=='':
                synopsis = chrome.find_elements_by_xpath('/html/body/div[3]/div[2]/div[3]/div/section[1]/p[2]')[0].text.split(' Fermer')[0]        
    except:
        synopsis = chrome.find_elements_by_class_name('pvi-productDetails-resume')[0].text
  
        
    charts = chrome.find_elements_by_class_name("pvi-product-scrating")
    c = [elt.text+ "\n"  for elt in charts]
    c = c[0].split("\n")
    repartition_note = c[1].split(" ")
    for j in range(len(repartition_note)):
        repartition_note[j] = int(repartition_note[j])
   
    dico_film = {"nom_film" : nom_film,
                 "type_production" : type_production,
                 "theme_production" :style_film,
                 "Synopsis" : synopsis, 
                 "Realisateur" : realisateur,  
                 "duree_film" : duree, 
                 "sortie_film" : date_de_sortie,
                 "acteurs_principaux" : acteurs,
                 "charts_SENSCRITIQUE" : {"note_moyenne" : float(c[0]), "nombre_critiques" : sum(repartition_note), "repartition_note":repartition_note}
        }
    
    return dico_film

In [57]:
#Commande qui prend beaucoup de temps(environ 2h qui va scrappé toutes les infos)
"""
lst_dico = []
for i in range(2013, 2020):
    chrome.get(liste_liens[i])
    for j in range(len(liste_nom_film[i-2013])):
        chrome.execute_script("window.scrollTo(0, document.body.scrollHeight);") 
        time.sleep(1)
        chrome.execute_script("scrollTo(0, 0);")
        lst_dico.append(recuperation_donnee(liste_nom_film[i-2013][j]))

        #ON RECUP TOUTES LES CHARTS LIE AU FILM

        #ON REPART SUR LA PAGE DE BASE ET ON REFAIT TOUT CE QU'ON VIENT DE VOIR POUR UN AUTRE FILM
        chrome.get(liste_liens[i])
        chrome.execute_script("window.scrollTo(0, document.body.scrollHeight);") 
        time.sleep(1)
        chrome.execute_script("scrollTo(0, 0);")
"""


'\nlst_dico = []\nfor i in range(2013, 2020):\n    chrome.get(liste_liens[i])\n    for j in range(len(liste_nom_film[i-2013])):\n        chrome.execute_script("window.scrollTo(0, document.body.scrollHeight);") \n        time.sleep(1)\n        chrome.execute_script("scrollTo(0, 0);")\n        lst_dico.append(recuperation_donnee(liste_nom_film[i-2013][j]))\n\n        #ON RECUP TOUTES LES CHARTS LIE AU FILM\n\n        #ON REPART SUR LA PAGE DE BASE ET ON REFAIT TOUT CE QU\'ON VIENT DE VOIR POUR UN AUTRE FILM\n        chrome.get(liste_liens[i])\n        chrome.execute_script("window.scrollTo(0, document.body.scrollHeight);") \n        time.sleep(1)\n        chrome.execute_script("scrollTo(0, 0);")\n'

In [9]:
#PROGRAMME DE TEST(Pour prouver que le scrapping marche) SENSCRITIQUE
def test_SENSCRITIQUE():
    liste_alea = liste_nom_film[r.randint(0,len(liste_nom_film)-1)]
    film  = liste_alea[r.randint(0,len(liste_alea)-1)]
    for i in range(len(liste_nom_film)):
        if liste_nom_film[i] == liste_alea:
            j=i

    chrome.get(liste_liens[2013+j])
    chrome.execute_script("window.scrollTo(0, document.body.scrollHeight);")   
    time.sleep(1)
    chrome.execute_script("scrollTo(0, 0);")

    return film, recuperation_donnee(film)

test_SENSCRITIQUE()

('Imitation Game',
 {'nom_film': 'Imitation Game',
  'type_production': 'Film',
  'theme_production': ['Biopic', ' drame', ' thriller'],
  'Synopsis': "En 1940, Alan Turing est chargé par le gouvernement britannique d'entraver le fonctionnement d'Enigma, la machine de cryptage allemande, réputée invulnérable.",
  'Realisateur': ['Morten Tyldum'],
  'duree_film': '1 h 54 min',
  'sortie_film': '14 novembre 2014',
  'acteurs_principaux': ['Benedict Cumberbatch',
   ' Keira Knightley',
   ' Matthew Goode'],
  'charts_SENSCRITIQUE': {'note_moyenne': 7.0,
   'nombre_critiques': 46688,
   'repartition_note': [72,
    127,
    305,
    815,
    2230,
    7160,
    16421,
    13738,
    4621,
    1199]}})

In [10]:
#TEST POUR LA SUITE A EXECUTER POUR POUVOIR TESTER LE SCRAPPING SUR IMDB
lst_dico = []
for i in range(5):
    a = test_SENSCRITIQUE()
    while a[1] in lst_dico:
        a = test_SENSCRITIQUE()
    lst_dico.append(a[1])
    
lst_dico 
    

[{'nom_film': "Why Don't You Play in Hell?",
  'type_production': 'Film',
  'theme_production': ['Action', ' comédie', 'drame'],
  'Synopsis': "Muto et Ikegami sont deux gangsters qui se détestent : l'un cherche un rôle de cinéma pour sa fille, l'autre est amoureux de cette dernière.",
  'Realisateur': ['Sion Sono'],
  'duree_film': '2 h 06 min',
  'sortie_film': '28 septembre 2013',
  'acteurs_principaux': ['Jun Kunimura',
   ' Fumi Nikaidô',
   " Shin'ichi Tsutsumi"],
  'charts_SENSCRITIQUE': {'note_moyenne': 7.5,
   'nombre_critiques': 2831,
   'repartition_note': [12, 26, 36, 67, 130, 301, 692, 974, 468, 125]}},
 {'nom_film': 'Les Gardiens de la galaxie Vol. 2',
  'type_production': 'Film',
  'theme_production': ['Action', ' science-fiction', ' comédie'],
  'Synopsis': 'Les Gardiens de la galaxie Vol. 2 nous entraîne dans la suite des aventures de cette drôle de bande de mercenaires aux confins du cosmos. Les Gardiens vont cette fois devoir lutter pour rester unis lorsqu’ils percen

# SCRAPPING IMDB

In [11]:
def c_film(film):
    return "_".join("_".join("_".join(film.split(' ')).split(":")).split('-'))

def indice_nom_film(nom_film, liste_film):
    for i in range(len(liste_film)):
        if nom_film == liste_film[i]['nom_film']:
            return i
        
def scrapping_IMDB(nom_film, liste_film, chemin_dossier_image):
    chrome.get("https://www.imdb.com/")
    inputElement = chrome.find_element_by_id("suggestion-search")
    inputElement.send_keys(nom_film)
    inputElement.submit() 
    time.sleep(1)
    
    j = indice_nom_film(nom_film, liste_film)
    annee = int(liste_film[j]['sortie_film'][-4:])
    indice = 0
    
    t = chrome.find_element_by_class_name('findSection').text.split('\n')
    for i in range(len(t)):
        if str(annee) in t[i] and "(TV Episode)" not in t[i] and "(TV Series)" not in t[i] and "(Video)" not in t[i] and "(Short)" not in t[i] and "(TV Mini-Series)" not in t[i]:
            indice = i
            break 
        elif str(annee+1) in t[i] and "(TV Episode)" not in t[i] and "(TV Series)" not in t[i] and "(Video)" not in t[i] and "(Short)" not in t[i] and "(TV Mini-Series)" not in t[i]:
            indice = i
            break 
        elif str(annee-1) in t[i] and "(TV Episode)" not in t[i] and "(TV Series)" not in t[i] and "(Video)" not in t[i] and "(Short)" not in t[i] and "(TV Mini-Series)" not in t[i]:
            indice = i
            break 
    
    link = chrome.find_element_by_link_text(chrome.find_element_by_class_name('findSection').text.split('\n')[indice].split(' (')[0])
    link.click()
    time.sleep(1)
    
    #On récupère l'image de la fiche et on la sauvegarde
    try: 
        a = chrome.find_element_by_xpath('//*[@id="title-overview-widget"]/div[1]/div[3]/div[1]/a/img')
    except:
        a = chrome.find_element_by_xpath('//*[@id="title-overview-widget"]/div[2]/div[1]/a/img')
    src = a.get_attribute('src')
    img = requests.get(src)
    image = Image.open(io.BytesIO(img.content))
 
    image.save(str(chemin_dossier_image)+'/'+str(c_film(nom_film))+'_poster.jpg')
    
    chrome.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    chrome.execute_script("scrollTo(0, 0);")
    
    a = chrome.find_element_by_class_name('ratings_wrapper')
    charts_IMDB = [a.text.split('\n')[0].split('/10')[0], a.text.split('\n')[1]]
    
    b = chrome.find_element_by_class_name('titleReviewBar')
    charts_METACRITIC = b.text.split("\n")[0]
    
    chrome.execute_script("window.scrollTo(0, document.body.scrollHeight);") 
    
    details = chrome.find_element_by_id('titleDetails')
    budget = None
    Cumulative_WW_Gross = None 
    location = None 
    for elm in details.text.split('\n'):
        if "Budget:" in elm:
            budget = "".join(elm.split('Budget:')[1]).split(' (estimated)')[0]
        elif "Cumulative Worldwide Gross:" in elm:
            Cumulative_WW_Gross = elm.split('Cumulative Worldwide Gross: ')[1]
        elif "Country:" in elm:
            location = elm.split('Country: ')[1]

    info_details = {'budget' : budget,
             'Cumulative_WW_Gross' : Cumulative_WW_Gross,
               'location' : location}
    liste_film[j]['charts_IMDB'] = charts_IMDB
    liste_film[j]['charts_METACRITIC'] = charts_METACRITIC
    liste_film[j]['autres_details'] = info_details

In [110]:
#Comme tout à l'heure cette boucle prend beaucoup de temps(environ 1h)
#Si dessous, des instructions pour tester si le scrapping marche réellement
"""
for i in range(len(lst_dico)):
    scrapping_IMDB(lst_dico[i]['nom_film'], lst_dico, 'static/poster')
"""
    

"\nfor i in range(len(lst_dico)):\n    scrapping_IMDB(lst_dico[i]['nom_film'], lst_dico, 'static/poster')\n"

In [12]:
#PROGRAMME DE TEST(Pour prouver que le scrapping marche) IMDB
import copy
film = lst_dico[r.randint(0,len(lst_dico)-1)]['nom_film']
i = indice_nom_film(film, lst_dico)
a = copy.deepcopy(lst_dico[i])
scrapping_IMDB(film, lst_dico, 'dossier_test_images')
a,lst_dico[i]

({'nom_film': 'Monstres Academy',
  'type_production': "Long-métrage d'animation",
  'theme_production': ['Animation', ' aventure', 'comédie'],
  'Synopsis': "Bob et Sulli sont de retour, mais maintenant nous les suivons à travers l’université où ils doivent grandir et s'unir pour devenir les meilleurs monstres du campus.",
  'Realisateur': ['Dan Scanlon'],
  'duree_film': '1 h 44 min',
  'sortie_film': '21 juin 2013',
  'acteurs_principaux': ['Billy Crystal', ' John Goodman', ' Helen Mirren'],
  'charts_SENSCRITIQUE': {'note_moyenne': 6.4,
   'nombre_critiques': 27848,
   'repartition_note': [92,
    198,
    445,
    1329,
    3461,
    7720,
    9335,
    4015,
    922,
    331]}},
 {'nom_film': 'Monstres Academy',
  'type_production': "Long-métrage d'animation",
  'theme_production': ['Animation', ' aventure', 'comédie'],
  'Synopsis': "Bob et Sulli sont de retour, mais maintenant nous les suivons à travers l’université où ils doivent grandir et s'unir pour devenir les meilleurs mo

# EXPORTATION DES DONNEES EN CSV

Et enfin on exporte les données que nous avons récupérer dans un csv

In [133]:
import pandas as pd
df = pd.DataFrame(lst_dico)
df.to_csv('data_FINAL.csv')